# Alpha to CT interactive viewer
Load mesh alpha/log_alpha, map to CT (T00), and browse slices.

In [ ]:
import os
import sys
from pathlib import Path

# Ensure repo root (containing GCN_SMS/) is on sys.path
cwd = Path.cwd().resolve()
repo_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "GCN_SMS").is_dir():
        repo_root = p
        break
if repo_root is None:
    raise RuntimeError("Could not find GCN_SMS package root.")
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from GCN_SMS.utils.alpha_slice_viewer import (
    interpolate_to_ct,
    interactive_slicer,
    mesh_to_ct_coords,
    interactive_mesh_slicer,
)

In [ ]:
# Case selection and paths (T00)
case_number = 2  # e.g., 1..10
epoch = 90
omega_u = 0.5
omega_alpha = 0.5
base_time = 'T00'

case_id = f'Case{case_number}Pack'
subj = f'case{case_number}'

mesh_npz = repo_root / f'checkpoints/mesh_results/{case_id}_epoch{epoch:03d}_omegaU{omega_u:.3f}_omegaA{omega_alpha:.3f}.npz'
ct_path = repo_root / f'data/Emory-4DCT/{case_id}/NIFTI/{subj}_{base_time}.nii.gz'
lung_mask_path = repo_root / f'data/Emory-4DCT/{case_id}/TotalSegment/{subj}_{base_time}/lung_combined_mask.nii.gz'
use_log_alpha = False  # training NPZ already stores alpha
# Optional: set env vars if you want to control threads
os.environ['OMP_NUM_THREADS'] = os.environ.get('OMP_NUM_THREADS', '4')

In [ ]:
# Interpolate alpha->E onto CT grid (CPU KDTree)
ct_vol, E_vol, aff = interpolate_to_ct(
    mesh_npz=mesh_npz,
    ct_path=ct_path,
    use_log_alpha=use_log_alpha,
    base_time=base_time,
    return_kpa=True,
    mask_path=lung_mask_path,
)

In [ ]:
# Interactive slicer: axis=2 for axial, 1 coronal, 0 sagittal (voxel field)
interactive_slicer(ct_vol, E_vol, axis=2, cmap='bwr', alpha=0.5)

In [ ]:
# Optional: slice the mesh nodes directly on top of CT
ct_vol_mesh, vox_coords, E_node, _ = mesh_to_ct_coords(
    mesh_npz=mesh_npz,
    ct_path=ct_path,
    use_log_alpha=use_log_alpha,
    base_time=base_time,
    return_kpa=True,
)
interactive_mesh_slicer(ct_vol_mesh, vox_coords, E_node, axis=2, cmap='bwr', alpha=0.8)